# Introduction to LangGraph

Welcome to your LangGraph learning journey! This notebook will introduce you to the fundamental concepts of LangGraph and help you set up your development environment.

## What is LangGraph?

LangGraph is a library for building stateful, multi-actor applications with LLMs. It extends LangChain by adding the ability to coordinate multiple chains (or actors) across multiple computational steps in a cyclic manner.

### Key Concepts:
- **State Machines**: LangGraph is built on the concept of state machines
- **Nodes**: Individual computational steps
- **Edges**: Connections between nodes that define the flow
- **State**: The data that flows through your graph

## Learning Objectives

By the end of this notebook, you will:
1. Understand what LangGraph is and when to use it
2. Set up your development environment
3. Create your first simple LangGraph application
4. Understand the basic components of a LangGraph workflow

## 1. Environment Setup

First, let's install the required packages and set up our environment.

In [ ]:
# Install required packages (run this cell if you haven't already)
# !pip install langgraph langchain langchain-openai python-dotenv

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from typing_extensions import TypedDict
import json

# Load environment variables
load_dotenv()

# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✅ Environment setup complete!")

## 2. Understanding LangGraph Components

Let's break down the main components of LangGraph:

### State

State is the data that flows through your graph. It's typically defined as a TypedDict:

In [ ]:
# Define our state structure
class AgentState(TypedDict):
    """The state of our agent."""
    messages: Annotated[list, "The messages in the conversation"]
    current_step: Annotated[str, "The current step in the process"]
    result: Annotated[str, "The final result"]

print("✅ State structure defined!")

### Nodes

Nodes are functions that process the state and return updated state:

In [ ]:
# Define a simple node function
def process_message(state: AgentState) -> AgentState:
    """Process the latest message in the conversation."""
    # Get the latest message
    latest_message = state["messages"][-1]
    
    # Generate a response using the LLM
    response = llm.invoke(f"Respond to: {latest_message.content}")
    
    # Update the state
    state["messages"].append(response)
    state["current_step"] = "processed"
    state["result"] = response.content
    
    return state

print("✅ Node function defined!")

## 3. Creating Your First LangGraph Application

Now let's create a simple graph that processes messages:

In [ ]:
# Create the graph
workflow = StateGraph(AgentState)

# Add the node
workflow.add_node("process_message", process_message)

# Set the entry point
workflow.set_entry_point("process_message")

# Add the end point
workflow.add_edge("process_message", END)

# Compile the graph
app = workflow.compile()

print("✅ Graph created and compiled!")

## 4. Running Your First Graph

Let's test our simple graph:

In [ ]:
# Create initial state
initial_state = {
    "messages": [
        {"role": "user", "content": "Hello! Can you tell me a joke?"}
    ],
    "current_step": "start",
    "result": ""
}

# Run the graph
result = app.invoke(initial_state)

print("🎉 Your first LangGraph application worked!")
print(f"\nFinal result: {result['result']}")
print(f"\nTotal messages: {len(result['messages'])}")

## 5. Understanding the Flow

Let's visualize what happened:

1. **Initial State**: We started with a user message
2. **Node Processing**: The `process_message` node processed the message
3. **State Update**: The state was updated with the LLM's response
4. **End**: The graph reached the END node

This is the simplest possible LangGraph application!

## 6. Exercise: Modify the Graph

Try modifying the graph to add another node that formats the response. Here's a template:

In [ ]:
# Exercise: Add a formatting node
def format_response(state: AgentState) -> AgentState:
    """Format the response to make it more readable."""
    # Your code here
    # Hint: Modify the result to add some formatting
    return state

# Create a new graph with both nodes
# Your code here

print("Try implementing the formatting node!")

## 7. Key Takeaways

### What we learned:
1. **State**: How to define and manage state in LangGraph
2. **Nodes**: How to create functions that process state
3. **Graphs**: How to connect nodes to create workflows
4. **Execution**: How to run and test your graphs

### Next Steps:
In the next notebook, we'll explore more complex state management patterns and learn how to create more sophisticated workflows.

## 8. Challenge

Try creating a graph that:
1. Takes a user question
2. Generates multiple possible answers
3. Picks the best one
4. Formats it nicely

This will help you understand how to chain multiple nodes together!

## Summary

Congratulations! You've successfully:
- ✅ Set up your LangGraph development environment
- ✅ Created your first LangGraph application
- ✅ Understood the basic components (State, Nodes, Graphs)
- ✅ Run a simple workflow

You're now ready to move on to more complex patterns in the next notebook!